# Fine-Tuning Mistral 7b v0.3 Model for Classification of Phishing Emails Using Unsloth Techniques

###Overview:

This project focuses on fine-tuning the Mistral 7b v0.3 model using Unsloth techniques for the classification of phishing emails. Beginning with the utilization of Unsloth for efficient fine-tuning, the project aims to optimize the training process by leveraging low-resource techniques, thereby reducing training times and improving resource efficiency.</br></br>

The next phase involves utilizing a phishing email dataset to train the model. This dataset provides labeled examples of email content categorized as safe or malicious, essential for training the model to accurately classify unseen emails based on their textual content and other relevant features.</br></br>

The SFT (Self-Supervised Fine-Tuning) Trainer is employed to facilitate effective model training. This includes configuring training parameters such as batch size, learning rate, and optimizer settings to maximize model performance and accuracy in distinguishing phishing emails from legitimate ones.</br></br>

Finally, the project demonstrates the model's inference capabilities for email classification. Users can input email text, and the model predicts whether the email is safe or potentially malicious in real-time, showcasing the practical application of the fine-tuned Mistral 7b v0.3 model in enhancing email security through automated classification.</br></br>

This comprehensive approach highlights the integration of advanced natural language processing techniques with practical cybersecurity applications, aiming to improve detection and response to phishing attacks using state-of-the-art language models.</br></br>

#Setup and Installation :

1. Imports torch and gets the CUDA device capability.
2. Installs the unsloth library from GitHub, suppressing the output to keep the notebook clean.

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers


Note : There are different methods to download unsloth for different GPUs. The following process is specific to the T4 GPU.

In [2]:
# Use this for older GPUs (V100, Tesla T4, RTX 20xx)
!pip install --no-deps xformers trl peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 13.7 MB/s eta 0:00:00


**Initialize FastLanguageModel**

*   Imports the FastLanguageModel from the unsloth library.
*   Sets the maximum sequence length for the model.
*   Configures the data type for computation based on the GPU type.
*   Enables 4-bit quantization to reduce memory usage, which can be disabled if not needed.



In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# dtype = torch.float16
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu124)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


**Loading the model**

This cell loads the Mistral AI model with 7 billion tokens, version 3, from the unsloth repository. It also defines the model and tokenizer in this cell.

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.15: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

### Configure Model
**Note**: We now add LoRA adapters so we only need to update 1 to 10% of all parameters!
* Model: Uses the previously loaded model.
* r: Rank for low-rank adaptation (e.g., 16).
* Target Modules: Specifies the modules to apply PEFT.
* LoRA Alpha: Scaling factor for LoRA.
* LoRA Dropout: Dropout rate for LoRA (optimized at 0).
* Bias: Bias configuration (optimized at "none").
* Gradient Checkpointing: Uses "unsloth" for efficient memory usage.
* Random State: Sets a seed for reproducibility.
* Use rslora: Option to use rank stabilized LoRA (disabled here).
* LoftQ Config: Option to use LoftQ (not used here).

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # max_seq_length = max_seq_length,
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Loading the dataset and preprocessing.
This cell loads the Phishing Email dataset from the Hugging Face repository zefang-liu/phishing-email-dataset. The dataset contains two columns: Email text and email type. The cell also includes preprocessing steps to prepare the data for model training and evaluation.

In [6]:
from datasets import load_dataset
dataset = load_dataset("zefang-liu/phishing-email-dataset", split = "train")

README.md:   0%|          | 0.00/616 [00:00<?, ?B/s]

Phishing_Email.csv:   0%|          | 0.00/52.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18650 [00:00<?, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['Unnamed: 0', 'Email Text', 'Email Type'],
    num_rows: 18650
})

In [8]:
#removing unncessory columns
dataset = dataset.remove_columns(['Unnamed: 0'])
dataset

Dataset({
    features: ['Email Text', 'Email Type'],
    num_rows: 18650
})

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(dataset[:5])

In [11]:
print(df)

                                          Email Text      Email Type
0  re : 6 . 1100 , disc : uniformitarianism , re ...      Safe Email
1  the other side of * galicismos * * galicismo *...      Safe Email
2  re : equistar deal tickets are you still avail...      Safe Email
3  \nHello I am your hot lil horny toy.\n    I am...  Phishing Email
4  software at incredibly low prices ( 86 % lower...  Phishing Email


## Format Phishing Email Dataset for the training
* Defining the prompt template
* Adding the EOS token to stop infinite generations.
* For training the model, we need all attributes in a single column. We have mapped the email text and email type into a text column using the prompt template and formatting function.

In [12]:
# Define the prompt template
phishing_prompt = """Below is an email body. Determine if the email is safe or phishing.

### Email Text:
{}

### Email Type:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# Define the formatting function
def formatting_prompts_func(examples):
    email_texts = examples["Email Text"]
    email_types = examples["Email Type"]
    texts = []
    for email_text, email_type in zip(email_texts, email_types):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = phishing_prompt.format(email_text, email_type) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/18650 [00:00<?, ? examples/s]

## Let's Train the Model
**Initialize SFTTrainer for Model Training**
* Model and Tokenizer: Uses model and tokenizer from unsloth.
* Training Dataset: Specifies dataset with formatted text in the "text" field.
* Training Settings: Configures batch size, gradient accumulation, learning rate, optimizer settings, and other training parameters.

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/18650 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/18650 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/18650 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/18650 [00:00<?, ? examples/s]

In [14]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.051 GB of memory reserved.


**Execute Model Training**

In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 18,650 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hongyan-workshop (hongyan-workshop-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


AttributeError: 'NoneType' object has no attribute 'attn_bias'

Final Memory State

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1017.9514 seconds used for training.
16.97 minutes used for training.
Peak reserved memory = 6.973 GB.
Peak reserved memory for training = 2.336 GB.
Peak reserved memory % of max memory = 47.281 %.
Peak reserved memory for training % of max memory = 15.839 %.


## Inference
Let's run the model! </br>
Give the mail body in the input box and it will show the Mail type whether it Safe mail or Phishing Mail.

In [ ]:
# Define the phishing email prompt
phishing_prompt = """Below is an email body. Determine if the email is safe or phishing.

### Email Text:
{}

### Email Type:
{}"""

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Example email text to classify
email_text_example = input("Please Enter mail body to Check if it is phishing or not : ")

# Prepare the inputs for the model
inputs = tokenizer(
[
    phishing_prompt.format(
        email_text_example,  # email text
        ""  # email type - leave this blank for generation!
    )
], return_tensors="pt").to("cuda")

# Generate the outputs using the model
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
print(tokenizer.batch_decode(outputs))

# For streaming inference
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Please Enter mail body to Check if it is phishing or not : Dear Customer,  Please find attached your monthly bank statement for June 2024. If you have any questions, please contact our support team.  Thank you for banking with us.  Best regards, Your Bank


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s>Below is an email body. Determine if the email is safe or phishing.\n\n### Email Text:\nDear Customer,  Please find attached your monthly bank statement for June 2024. If you have any questions, please contact our support team.  Thank you for banking with us.  Best regards, Your Bank\n\n### Email Type:\nSafe Email</s>']
<s>Below is an email body. Determine if the email is safe or phishing.

### Email Text:
Dear Customer,  Please find attached your monthly bank statement for June 2024. If you have any questions, please contact our support team.  Thank you for banking with us.  Best regards, Your Bank

### Email Type:
Safe Email</s>


#Saving the finetuned model

In [ ]:
model.save_pretrained("mistral_v3_phishing") # Local saving
tokenizer.save_pretrained("mistral_v3_phishing")

('mistral_v3_phishing/tokenizer_config.json',
 'mistral_v3_phishing/special_tokens_map.json',
 'mistral_v3_phishing/tokenizer.model',
 'mistral_v3_phishing/added_tokens.json')

**Use saved model to generate inference**

In [ ]:
if False:
  from unsloth import FastLanguageModel
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = "mistral_v3_phishing", #model which i have saved.
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
  )

  FastLanguageModel.for_inference(model) #Enable 2x faster inference



In [ ]:
#lets define the phishing prompt again

phishing_prompt = """ Below is an email body. Determine if the email is safe or phishing.

### Email Text:
{}

### Email Type:
{}"""

In [ ]:
#Take the input of Example email to classify whether it is safe or phshing
email_text_example = input("Enter the Email body to classify whether it is safe or phishing: ")

#lets prepare the input for the model
inputs = tokenizer(
    [
        phishing_prompt.format(
            email_text_example,  #email text
            "" #email_type  leave this blank for generation.
        )
    ], return_tensors="pt").to("cuda")


#generate the output using the model
outputs = model.generate(**inputs,max_new_tokens=64,use_cache=True)
print(tokenizer.batch_decode(outputs))


# For streaming inference
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

Enter the Email body to classify whether it is safe or phishing: Dear Valued Customer,  Thank you for your recent purchase with us. Your invoice is attached to this email. If you have any questions about your order, please contact our support team.  [Download Invoice](http://phishing-link.com)  We appreciate your business.  Best regards, Customer Support Team


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an email body. Determine if the email is safe or phishing.\n\n### Email Text:\nDear Valued Customer,  Thank you for your recent purchase with us. Your invoice is attached to this email. If you have any questions about your order, please contact our support team.  [Download Invoice](http://phishing-link.com)  We appreciate your business.  Best regards, Customer Support Team\n\n### Email Type:\nPhishing Email</s>']
<s> Below is an email body. Determine if the email is safe or phishing.

### Email Text:
Dear Valued Customer,  Thank you for your recent purchase with us. Your invoice is attached to this email. If you have any questions about your order, please contact our support team.  [Download Invoice](http://phishing-link.com)  We appreciate your business.  Best regards, Customer Support Team

### Email Type:
Phishing Email</s>
